In [1]:
import os
import json 
import logging

logging.basicConfig(
    filename='log/app.log',            # Specify the log file name
    level=logging.DEBUG,           # Set the log level (DEBUG, INFO, WARNING, ERROR, CRITICAL)
    format='%(asctime)s - %(levelname)s - %(message)s'  # Set the log format
)

# Load the environment configuration JSON data
json_path = 'env_config.json'
with open(json_path, 'r') as file:
    env_config = json.load(file)

hf_home = env_config['HF_HOME']
# Set the HF_HOME environment variable
os.environ['HF_HOME'] = hf_home
# Set the access token to huggingface hub
access_token = env_config['access_token']
os.environ['HUGGINGFACE_HUB_TOKEN'] = access_token

In [2]:
import transformers 
print(transformers.__version__)

from transformers import pipeline
import torch

from accelerate import Accelerator
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel
from transformers import LlamaTokenizerFast
import torch.nn.functional as F


accelerator = Accelerator()
device = accelerator.device

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_id = "meta-llama/Meta-Llama-3-8B"  # non-instruct version

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    token=access_token,
)

tokenizer = LlamaTokenizerFast.from_pretrained(model_id, token=access_token)

/opt/crc/c/conda/23.5.2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.41.0


Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.92s/it]
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'LlamaTokenizerFast'.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [20]:
from datasets import load_dataset
from torch.utils.data import DataLoader
# from llmexp.helper import LlmExpHelper
class LlmExpHelper:
    def __init__(self, tokenizer, model, device):
        self.tokenizer = tokenizer
        self.model = model
        self.device = device
    
    def get_collate_fun(self):
        return lambda examples: self.collate_fn(examples)

    def collate_fn(self, examples):
        def num_words(x):
            return len(x.split())
        def get_first_k_words(x, k):
            return ' '.join(x.split()[:k])
        tokenizer = self.tokenizer
        max_len = 256 # characters limit other than token limit
        texts = [example['sentence'] for example in examples]
        # texts = [example['text'] for example in examples]
        texts = [text if num_words(text) <= max_len else get_first_k_words(text, max_len) for text in texts]
        labels = [example['label'] for example in examples]
        messages_lambda = lambda texts: [
            {"role": "system", "content": "You are a chatbot for sentiment analysis. You can help users with their questions via concise responses of POSITIVE, or NEGATIVE."},
            # {"role": "system", "content": "You are a chatbot for sentimate analysis."},
            {"role": "user", "content": texts},
        ]
        messages = list(map(messages_lambda, texts))

        messages_with_template_applied = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
        )
        batch = tokenizer(
                    messages_with_template_applied,
                    add_special_tokens=False,
                    padding=True,
                    return_tensors="pt",
                    )
        
        # find the template boundaries
        text_lens = [len(tokenizer.encode(text)) - 1 for text in texts] # note that the tokenizer.encode adds the eos token
        text_lens_tensor = torch.tensor(text_lens, dtype=torch.long)
        
        def apply_mask(mask_tensor, text_lens_tensor):
            batch_size, seq_len = mask_tensor.shape
            for i in range(batch_size):
                text_len = text_lens_tensor[i].item()
                mask_tensor[i, -text_len-5:-5] = 0
            return 1- mask_tensor

        mask_tensor = apply_mask(torch.ones_like(batch['input_ids']), text_lens_tensor)

        batch['context_mask'] = mask_tensor
        
        return batch

# imdb = load_dataset("imdb")
# train_ds = imdb['train']
# from datasets import load_dataset

ds = load_dataset("stanfordnlp/sst2")
train_ds = ds['train']
llm_exp_helper = LlmExpHelper(tokenizer, model, device)
collate_fn = llm_exp_helper.get_collate_fun()

# Define batch size here!
batch_size = 16
train_dataloader = DataLoader(train_ds, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)

In [4]:
train_ds[1]

{'text': '"I Am Curious: Yellow" is a risible and pretentious steaming pile. It doesn\'t matter what one\'s political views are because this film can hardly be taken seriously on any level. As for the claim that frontal male nudity is an automatic NC-17, that isn\'t true. I\'ve seen R-rated films with male nudity. Granted, they only offer some fleeting views, but where are the R-rated films with gaping vulvas and flapping labia? Nowhere, because they don\'t exist. The same goes for those crappy cable shows: schlongs swinging in the breeze but not a clitoris in sight. And those pretentious indie movies like The Brown Bunny, in which we\'re treated to the site of Vincent Gallo\'s throbbing johnson, but not a trace of pink visible on Chloe Sevigny. Before crying (or implying) "double-standard" in matters of nudity, the mentally obtuse should take into account one unavoidably obvious anatomical difference between men and women: there are no genitals on display when actresses appears nude, 

In [22]:
from llmexp.model6 import MaskGeneratingModel

mask_gen_model = MaskGeneratingModel(hidden_size=4096, mlp_hidden_dim=1024, mlp_bottleneck_dim=768, mlp_num_blocks=2)
mask_gen_model.to(device)


from tqdm import tqdm

# Set pad_token_id if it is not set
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

pad_token_id = tokenizer.pad_token_id

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

optimizer = torch.optim.Adam(mask_gen_model.parameters(), lr=1e-5)

In [23]:
mask_gen_model.train()
for epoch in range(1):
    pbar = tqdm(train_dataloader)
    for idx, data in enumerate(pbar):
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        context_mask = data['context_mask'].to(device)
        # get generated texts
        gen_outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=128,
            eos_token_id=terminators,
            pad_token_id=tokenizer.pad_token_id,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
            return_dict_in_generate=True,
            output_scores=True,
        )
        gen_tokens = gen_outputs.sequences
        pad_length = gen_tokens.size(1) - input_ids.size(1)
        # get the attention mask for the generated tokens, and also mask the padding tokens
        gen_attention_mask = F.pad(attention_mask, (0, pad_length), mode='constant', value=1)
        # (gen_tokens != pad_token_id).long() is the tokens mask, 1 for real tokens and 0 for padding tokens
        unpaded_token_mask = (gen_tokens != pad_token_id).long()
        unpaded_token_mask[:, :-pad_length] = 1
        gen_attention_mask = gen_attention_mask * unpaded_token_mask
        # get the response mask, which is the mask for the generated tokens (the user inputs are masked with 0)
        response_mask = gen_attention_mask.clone()
        response_mask[:, :-pad_length] = 0 # TODO: 有问题. 有问题吗？

        context_mask = F.pad(context_mask, (0, pad_length), mode='constant', value=0)

        # Get the last hidden state for the prompt + response sequence
        with torch.no_grad():
            full_outputs = model(input_ids=gen_tokens, attention_mask=gen_attention_mask, output_hidden_states=True, return_dict=True)
            last_hidden_state = full_outputs.hidden_states[-1]
            last_hidden_state = last_hidden_state.float()
        
        mask_logits = mask_gen_model(last_hidden_state)

        mask_gen_outputs = mask_gen_model.loss_func(model, gen_tokens, gen_attention_mask, context_mask, mask_logits, response_mask, 
                                                                           num_samples=5)
        loss, reward_loss, mask_loss, mask_mean, mean_reward = mask_gen_outputs['loss'], mask_gen_outputs['reward_loss'], mask_gen_outputs['mask_loss'], mask_gen_outputs['mask_mean'], mask_gen_outputs['mean_reward']
        log = (f"Epoch {epoch+1}, Step {idx+1}: Loss = {loss.item():.4f}, " 
                             f"Reward Loss = {reward_loss.item():.4f}, "
                             f"Mean Reward = {mean_reward.item():.4f},"
                             f"Mask_loss = {mask_loss.item():.4f} "
                             f"mask_mean = {mask_mean.item():.4f}"
        )
        pbar.set_description(log)
        logging.debug(log)
        # Train the model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #                      )
        # if idx % 10 == 0:
        #     print()
        if idx % 200 == 0 and idx != 0:
            torch.save(mask_gen_model.state_dict(), f'saved_model/mask_gen_model_{epoch}_{idx}.pth') 
            print()
            # break

Epoch 1, Step 56: Loss = 0.3094, Reward Loss = 0.3094, Mean Reward = 0.6607,Mask_loss = 0.0107 mask_mean = 0.2315:   1%|▏         | 56/4210 [00:46<58:04,  1.19it/s]  


KeyboardInterrupt: 

In [ ]:
loss

tensor(1.8184, device='cuda:0', grad_fn=<MeanBackward0>)

In [32]:
import numpy as np
idx = 0
from captum.attr import visualization as viz
import torch.nn.functional as F

mask_gen_model.eval()

# tokens = tokenizer.convert_ids_to_tokens(gen_tokens[idx])
# texts = "This movie was the best movie I have ever seen! some scenes were ridiculous, but acting was great."
# texts = "I did not like this movie. Some of the actors were good, but overall the movie was boring."
# texts = "I hate that I love you."
# texts = "I don't like this movie."
# texts = "I really love this film."
texts = "I really love this film. The acting was great, and the story was amazing. I would recommend this movie to everyone."
# texts = "I don't like this movie. The acting was terrible, and the story was boring. I would not recommend this movie to anyone."
messages_lambda = lambda texts: [
            {"role": "system", "content": "You are a chatbot for sentiment analysis. You can help users with their questions via concise responses of POSITIVE, or NEGATIVE."},
            # {"role": "system", "content": "You are a chatbot for sentimate analysis."},
            {"role": "user", "content": texts},
        ]
messages = messages_lambda(texts)
messages_with_template_applied = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
        )

# test_text = [{"text": texts, "label": None}]
test_text = [{"sentence": texts, "label": None}]
test_inputs = collate_fn(test_text).to(device)

# test_inputs = next(iter(train_dataloader)).to(device)
tokens = tokenizer.convert_ids_to_tokens(test_inputs['input_ids'][idx])

# generate the answer for the test inputs
gen_outputs = model.generate(
            input_ids=test_inputs['input_ids'],
            attention_mask=test_inputs['attention_mask'],
            max_new_tokens=128,
            eos_token_id=terminators,
            pad_token_id=tokenizer.pad_token_id,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
            return_dict_in_generate=True,
            output_scores=True,
        )
input_ids = test_inputs['input_ids']
attention_mask = test_inputs['attention_mask']
gen_tokens = gen_outputs.sequences
pad_length = gen_tokens.size(1) - input_ids.size(1)
# get the attention mask for the generated tokens, and also mask the padding tokens
gen_attention_mask = F.pad(attention_mask, (0, pad_length), mode='constant', value=1)
context_mask = F.pad(test_inputs['context_mask'], (0, pad_length), mode='constant', value=0)
# (gen_tokens != pad_token_id).long() is the tokens mask, 1 for real tokens and 0 for padding tokens
unpaded_token_mask = (gen_tokens != pad_token_id).long()
unpaded_token_mask[:, :-pad_length] = 1
gen_attention_mask = gen_attention_mask * unpaded_token_mask

with torch.no_grad():
    # prompt_outputs = model(input_ids=test_inputs['input_ids'], attention_mask=test_inputs['attention_mask'], output_hidden_states=True, return_dict=True)
    prompt_outputs = model(input_ids=gen_tokens, attention_mask=gen_attention_mask, output_hidden_states=True, return_dict=True)

    last_hidden_state = prompt_outputs.hidden_states[-1].float()
    mask_logits = mask_gen_model(last_hidden_state)

# Function to clean tokens
def clean_token(token):
    # Remove special characters like "Ġ" or "Ċ"
    return token.replace("Ġ", "").replace("Ċ", "").replace("Ġ", "").replace("Ċ", "").replace("Ġ", "")

# Apply cleaning to each token
tokens = tokenizer.convert_ids_to_tokens(gen_tokens[idx])
tokens = [clean_token(token) for token in tokens]

def normalize_except_zeros(array):
    # Create a mask to identify non-zero elements
    mask = array != 0
    
    # Extract non-zero elements
    non_zero_elements = array[mask]
    
    # Normalize non-zero elements
    min_val = np.min(non_zero_elements)
    max_val = np.max(non_zero_elements)
    # normalized_non_zero_elements = (non_zero_elements - min_val) / (max_val - min_val)
    # get the 50% persentile
    # min_val = np.percentile(non_zero_elements, 50)
    normalized_non_zero_elements = (non_zero_elements - min_val) / (max_val - min_val)
    
    # Create a copy of the original array to preserve zero values
    normalized_array = np.copy(array)
    
    # Assign normalized values back to the corresponding positions
    normalized_array[mask] = normalized_non_zero_elements
    
    return normalized_array

expl = (torch.sigmoid(mask_logits) * context_mask)[idx]
expl = F.pad(expl, (0, len(tokens) - expl.size(0)), mode='constant', value=0)
expl_raw = expl.detach().cpu().numpy()
# expl = (expl - 0) / (expl_raw.max(axis=-1) - 0)

# expl = expl_raw
expl = normalize_except_zeros(expl_raw)



# expl = response_mask[idx]


vis_data_records = [viz.VisualizationDataRecord(
                                expl,
                                0,
                                0,
                                0,
                                0,
                                1,       
                                tokens,
                                1)]
                            
viz.visualize_text(vis_data_records)

In [17]:
# Set print options for precision
torch.set_printoptions(precision=4)
np.set_printoptions(precision=4)
print(expl)

[0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
 0.0000e+00 2.4742e-02 6.0288e-01 4.3501e-01 3.8011e-01 7.7301e-02
 4.1161e-04 9.5759e-05 8.4586e-01 4.7034e-01 7.6526e-01 9.9955e-03
 4.9102e-01 5.7602e-01 6.6518e-01 8.1205e-01 6.8847e-01 1.8136e-03
 1.8811e-04 9.5622e-01 9.5475e-01 2.4792e-01 1.1541e-01 1.3485e-01
 5.3585e-03 1.0000e+00 0.0000e+00 5.5876e-05 0.0000e+00 0.0000e+00
 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]


In [33]:
expl_raw

array([0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 4.6668e-01, 1.9327e-02, 4.8100e-01,
       7.2008e-01, 9.0109e-01, 1.5161e-01, 4.6525e-02, 2.6054e-02,
       4.0412e-04, 5.0007e-01, 2.5481e-02, 1.1995e-01, 5.5544e-02,
       6.3519e-01, 1.4506e-02, 6.9871e-01, 1.0421e-01, 3.4356e-01,
       1.2204e-02, 9.5965e-01, 3.3547e-01, 2.1295e-01, 2.8386e-01,
       6.4033e-01, 2.8961e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
      dtype=float32)

In [ ]:
mask_prob = torch.sigmoid(mask_logits)
(mask_prob * context_mask).sum(-1) / context_mask.sum(dim=-1)

tensor([0.1241, 0.2500, 0.7787, 0.0909, 0.1908, 0.1361, 0.3316, 0.5104, 0.2696,
        0.1001, 0.5931, 0.0812, 0.4139, 0.3334, 0.4853, 0.2987],
       device='cuda:0')

In [ ]:
mask_prob

tensor([[9.9998e-01, 1.3412e-02, 2.1168e-05, 7.2758e-04, 9.5831e-01, 2.8712e-06,
         1.9530e-03, 6.8251e-04, 9.6814e-01, 7.6582e-04, 2.2164e-06, 1.7009e-02,
         2.4596e-09, 1.4519e-04, 3.0463e-06, 7.9092e-04, 4.0336e-06, 5.2874e-05,
         1.4755e-07, 5.3417e-09, 1.0859e-07, 8.2496e-07, 1.0534e-05, 2.1604e-06,
         6.6980e-10, 1.8266e-01, 5.4030e-07, 4.0081e-02, 3.5185e-04, 3.1933e-01,
         3.2836e-09, 1.5197e-06, 8.2054e-01, 8.0839e-01, 5.4567e-01, 4.8829e-01,
         1.0000e+00, 4.3639e-05, 3.5097e-07, 3.8482e-09, 1.9672e-04, 1.1242e-07,
         2.7982e-13, 5.4683e-09, 4.5730e-01, 9.5505e-01, 6.7815e-04, 7.4368e-02,
         9.9924e-01, 6.6332e-03, 1.0508e-08, 1.8774e-01]], device='cuda:0')

In [ ]:
test_inputs['input_ids'][idx]

tensor([128000, 128006,   9125, 128007,    271,   2675,    527,    264,   6369,
          6465,    369,  27065,   6492,     13,   1472,    649,   1520,   3932,
           449,    872,   4860,   4669,  64694,  14847,    315,  27592,  45450,
            11,    477,  85165,  24093,     13, 128009, 128006,    882, 128007,
           271,   2028,   5818,    574,    279,   1888,   5818,    358,    617,
          3596,   3970,      0,   1063,  16451,   1051,  27873,     11,    719,
         15718,    574,   2294,     13, 128009, 128006,  78191, 128007,    271],
       device='cuda:0')

In [ ]:
tokenizer.convert_ids_to_tokens(271)

'ĊĊ'

[0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.6432 0.8566 0.5179
 0.2417 0.     0.1211 0.3355 0.618  0.5345 0.1401 0.3401 0.4729 0.3531
 0.661  0.7049 0.0297 0.1724 0.9905 1.     0.1606 0.1107 0.2363 0.2891
 0.116  0.0777 0.     0.     0.     0.     0.     0.     0.     0.    ]


In [ ]:
(torch.sigmoid(mask_logits) * context_mask)[idx]

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.8527, 0.1770, 0.1719, 0.1660, 0.1613, 0.1549, 0.1581, 0.1622, 0.1666,
        0.1635, 0.1568, 0.1589, 0.1618, 0.1634, 0.1668, 0.1647, 0.1546, 0.1576,
        0.1788, 0.1779, 0.1592, 0.1581, 0.1607, 0.1619, 0.1586, 0.1583, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
       device='cuda:0')

In [ ]:
torch.sigmoid(mask_logits)

tensor([[0.4822, 0.4821, 0.4816,  ..., 0.4920, 0.4874, 0.4901],
        [0.4843, 0.4851, 0.4855,  ..., 0.4941, 0.4861, 0.4891],
        [0.4785, 0.4805, 0.4805,  ..., 0.4918, 0.4823, 0.4864],
        ...,
        [0.4753, 0.4758, 0.4761,  ..., 0.4847, 0.4761, 0.4802],
        [0.4876, 0.4883, 0.4882,  ..., 0.4887, 0.4880, 0.4943],
        [0.4843, 0.4851, 0.4852,  ..., 0.4946, 0.4853, 0.4947]],
       device='cuda:0')

In [ ]:
mask_gen_model

MaskGeneratingModel(
  (explain_map): MLP(
    (input_layer): Linear(in_features=4096, out_features=1024, bias=True)
    (attention_layers): ModuleList(
      (0-1): 2 x MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
      )
    )
    (layers): ModuleList(
      (0-1): 2 x Sequential(
        (0): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (1): PReLU(num_parameters=1)
        (2): Linear(in_features=1024, out_features=1024, bias=True)
      )
    )
    (output_layer): Linear(in_features=1024, out_features=1, bias=True)
  )
)

In [ ]:
print(tokens[35])
print(expl[35])

<|end_header_id|>
0.0


In [ ]:
texts = "This movie was the best movie I have ever seen! some scenes were ridiculous, but acting was great."
# texts = "I really didn't like this movie. Some of the actors were good, but overall the movie was boring."
# texts = "I hate that I love you."
# texts = "I don't like this movie."
# texts = "I really love this film."
messages_lambda = lambda texts: [
    {"role": "system", "content": "You are a chatbot for sentimate analysis. You can help users with their questions via concise responses of POSITIVE, or NEGATIVE."},
    # {"role": "system", "content": "You are a chatbot for sentimate analysis."},
    {"role": "user", "content": texts},
]
messages = messages_lambda(texts)
messages_with_template_applied = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
        )
print(messages_with_template_applied)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a chatbot for sentimate analysis. You can help users with their questions via concise responses of POSITIVE, or NEGATIVE.<|eot_id|><|start_header_id|>user<|end_header_id|>

This movie was the best movie I have ever seen! some scenes were ridiculous, but acting was great.<|eot_id|><|start_header_id|>assistant<|end_header_id|>


